**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [12]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [46]:
ratings_agg = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'rating' : 'count'})
ratings_agg.head()

timestamp             rating
               min         max  count
userId                               
1       1260759108  1260759205     20
2        835355395   835356246     76
3       1298861589  1298932787     51
4        949778714   949982274    204
5       1163373044  1163375145    100

In [58]:
ratings_agg['delta'] = ratings_agg.timestamp['max'] - ratings_agg.timestamp['min']
ratings_agg['delta']

userId
1            97
2           851
3         71198
4        203560
5          2101
         ...   
667        1014
668         282
669         685
670     2162705
671    11283984
Name: delta, Length: 671, dtype: int64

In [61]:
ratings_agg2 = ratings_agg[ratings_agg.rating['count'] > 100]
ratings_agg2.head()

timestamp             rating      delta
               min         max  count           
userId                                          
4        949778714   949982274    204     203560
8       1154389340  1154474527    116      85187
15       997937239  1469330735   1700  471393496
17      1127468587  1127476640    363       8053
19       855190091   855195373    423       5282

In [65]:
ratings_agg2['delta'].mean()

40080507.4496124

**Задание  2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [91]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [67]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [68]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [69]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [103]:
full = rzd.merge(auto, how = 'outer', on = ['client_id']).merge(air, how = 'outer', on = ['client_id'])
full

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,NaN,NaN
112,2810.0,NaN,NaN
113,10283.0,57483.0,NaN
114,5774.0,83.0,NaN
115,981.0,912.0,81.0
116,NaN,4834.0,4.0
117,NaN,98.0,13.0
118,NaN,NaN,173.0


In [104]:
full2 = client_base.merge(rzd, how = 'outer', on = ['client_id']).merge(auto, how = 'outer', on = ['client_id']).merge(air, how = 'outer', on = ['client_id'])
full2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [105]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [106]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [107]:
(
    visits
    .pivot_table(index='date', columns = 'source',aggfunc='sum',values='visits')
    .reset_index() #возвращает индекс для применения merge
    .merge(orders, how='left')
)

,date,organic,paid,orders
0,2019-11-01,16825.0,1952.0,198
1,2019-11-02,21890.0,376.0,225
2,2019-11-03,19509.0,NaN,201


Ответ:

1. Полагаю, что нужно осуществлять привязку через user_id: у каждого обращения в интернет-магазин и у каждой покупки есть iser_id пользователя (по примеру датасета с выставлением оценок к фильмам). Сначала надо сделать сводные таблицы по user_id для таблицы с визитами/покупками и для таблицы с координатами, чтобы избавиться от дубликатов. Затем нам на выручку придёт merge, датасет с координатами мы будем прикреплять к таблице визитов+покупок (как в строке ноутбука выше, но вместо даты - user_id) через указатель user_id данного визита/покупки, или, если есть такая информация - через указатель времени совершения визита-покупки с одной строны и времени получения координаты - с другой стороны.  

2. При осуществлении действий из п. 1 удалённость координат одного и того же пользователя никак не отразится на вопросе расчетах пункта 1.

3. Если сопоставить координаты с ближайшими населёнными пунктами, то можно получить статистику активности визитов-покупок по данным географическим объектам.